In [ ]:
import napari
import numpy as np
from aicsimageio import AICSImage

import pandas as pd
from skimage import measure


import tifffile as tf
import matplotlib.pyplot as plt
import xarray as xr
from tifffile import tifffile
import tifftools
import os
from matplotlib import pyplot as plt
from os.path import sep
from skimage import io
from PIL import Image
import imageio


from skimage.measure import regionprops_table

import seaborn as sns

import czifile
from czifile import CziFile

import math
import matplotlib.colors as mcolors


In [ ]:
epg_fly = '/Users/sansachen/Downloads/lab/confocal/1_epg-glucla/05282024/4dayold1.czi'
fn=epg_fly

In [ ]:
# 1) Read raw array and its axes string
with CziFile(fn) as czi:
    axes = list(czi.axes)   # e.g. ['X','Y','C','Z','T']
    arr  = czi.asarray()    # e.g. shape (512, 512, 3, 104, 1)

print("Raw axes:", axes, "→ shape:", arr.shape)

# 2) Collapse any singleton dims not in CZYX
for dim in axes.copy():
    idx = axes.index(dim)
    if dim not in ('C','Z','Y','X') and arr.shape[idx] == 1:
        arr = np.take(arr, indices=0, axis=idx)
        axes.pop(idx)

print("After collapse:", axes, "→ shape:", arr.shape)
# Expect axes == ['X','Y','C','Z']

# 3) Permute from (X,Y,C,Z) → (C,Z,Y,X)
order = [axes.index(d) for d in ('C','Z','Y','X')]
arr   = np.transpose(arr, order)  # → shape (3, 104, 512, 512)
print("Reordered to CZYX:", arr.shape)

# 4) Split channels
red_channel   = arr[0]   # shape (Z, Y, X)
green_channel = arr[1]   # shape (Z, Y, X)                                              # add more if have blue channel!



In [ ]:
# i assumed that step (5)-(8) weren't used ever, and only (1)-(4) are useful and necessary to run the rest of codes.

# 5) Compute your MIPs
#   Red → attenuated MIP
attenuation = 0.05
z = np.arange(red_channel.shape[0])[:, None, None]
weights = np.exp(-attenuation * z)
mip_red   = np.max(red_channel * weights, axis=0)

#   Green → standard MIP
mip_green = np.max(green_channel, axis=0)

# 6) Normalize & gamma-correct
def norm_gamma(img, vmin, vmax, gamma):
    clipped = np.clip(img, vmin, vmax)
    normed  = (clipped - vmin) / (vmax - vmin)
    return normed ** (1.0 / gamma)

r = norm_gamma(mip_red,   16,  97, 0.83)
g = norm_gamma(mip_green, 33, 135, 1.0)

# 7) Colorize & blend
h, w = r.shape
red_rgb = np.zeros((h, w, 3))
red_rgb[..., 2] = r      # “bop blue” → blue channel

grn_rgb       = np.zeros((h, w, 3))
grn_rgb[..., 0] = g      # yellow → R+G
grn_rgb[..., 1] = g

composite = np.clip(red_rgb + grn_rgb, 0, 1)

# 8) Save as high-res SVG & EPS
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(composite, interpolation='nearest')
ax.axis('off')

for fmt in ('svg','eps'):
    fname = f'mip_composite.{fmt}'
    fig.savefig(
        fname,
        format=fmt,
        bbox_inches='tight',
        pad_inches=0,
        dpi=800    # bump to 1000+ if you need even more pixels
    )

In [ ]:
# open image in napari to draw labels:
viewer = napari.Viewer()

# Add the red channel
viewer.add_image(
    red_channel, 
    name='568 Channel',
    colormap='red',
    #contrast_limits=(100, 2500), # adjust this if needed
    blending='additive'
)
# Add the green channel
viewer.add_image(
    green_channel, 
    name='488 Channel',
    colormap='green',
    #contrast_limits=(50, 800), # adjust this if needed
    blending='additive'
)

napari.run()

In [ ]:
# for captuing a svg in **both channels** AT ONE SLICE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# --- 1) extract the single slice ---
z = 7
red_slice   = red_channel[z]    # shape (Y, X)
green_slice = green_channel[z]  # shape (Y, X)

# --- 2) normalize & clip to your contrast limits ---
def normalize(img, vmin, vmax):
    return np.clip((img - vmin) / (vmax - vmin), 0, 1)

r_norm = normalize(red_slice,   0,  61)
g_norm = normalize(green_slice, 0,  59)

# --- 3) map to your colors ---
# “bop blue” can be approximated—replace the hex with your exact if you have it
bop_blue_rgb = mcolors.to_rgb("#0090FF")
red_rgb      = np.array(mcolors.to_rgb("red"))

# expand to RGB images
h, w = r_norm.shape
r_img = r_norm[..., None] * bop_blue_rgb
g_img = g_norm[..., None] * red_rgb

# --- 4) additive composite & clip ---
composite = np.clip(r_img + g_img, 0, 1)

# --- 5) save as SVG (and EPS if you like) ---
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(composite, interpolation='nearest')
ax.axis('off')


for fmt in ('svg','eps'):
    fig.savefig(
        f'slice_z{z:02d}_composite.{fmt}',
        format=fmt,
        bbox_inches='tight',
        pad_inches=0,
        dpi=800
    )

In [ ]:
# for captuing a svg in **ONE channel** AT ONE SLICE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# tdt channel only

z = 7
red_slice = red_channel[z]   # shape (Y, X)

# 2) normalize to your contrast limits
def normalize(img, vmin, vmax):
    return np.clip((img - vmin) / (vmax - vmin), 0, 1)

r_norm = normalize(red_slice, 0, 61)

# 3) map to “bop blue”
bop_blue_rgb = mcolors.to_rgb("#0090FF")  
h, w = r_norm.shape
red_img = r_norm[..., None] * bop_blue_rgb  # shape (Y, X, 3)

# 4) that *is* your composite
composite = red_img

# 5) save as high-res SVG/EPS
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(composite, interpolation='nearest')
ax.axis('off')

for fmt in ('svg','eps'):
    fig.savefig(
        f'slice_z{z:02d}_red_only.{fmt}',
        format=fmt,
        bbox_inches='tight',
        pad_inches=0,
        dpi=800
    )

In [ ]:
# for captuing a svg in **ONE channel** AT ONE SLICE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# just to a different channel using a different color compared to the above cell
# gfp channel only:

# 1) pick your slice
z = 7
green_slice = green_channel[z]   # shape (Y, X)

# 2) normalize to your contrast limits
def normalize(img, vmin, vmax):
    return np.clip((img - vmin) / (vmax - vmin), 0, 1)

g_norm = normalize(green_slice, 0, 59)

# 3) map to red
red_rgb = mcolors.to_rgb("red")
h, w = g_norm.shape
green_img = g_norm[..., None] * red_rgb  # shape (Y, X, 3)

# 4) that is your image
composite = green_img

# 5) save as high-res SVG/EPS
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(composite, interpolation='nearest')
ax.axis('off')

for fmt in ('svg','eps'):
    fig.savefig(
        f'slice_z{z:02d}_green_only.{fmt}',
        format=fmt,
        bbox_inches='tight',
        pad_inches=0,
        dpi=800   # raise to 800–1000 for even higher res
    )